In [1]:
import pandas as pd
import numpy as np

In [11]:
data = pd.read_csv('.\datasets\hfi_cc_2018.csv')
#data = data[[
#'year',
#'countries',
#'region',
#'pf_rank',
#'pf_score',
#'pf_rol_civil',
#'pf_ss_homicide',
#'pf_ss_disappearances',
#'pf_movement',
#'pf_religion_estop',
#'pf_association_political_establish',
#'pf_association',
#'pf_expression_internet',
#'pf_identity_divorce',
#'ef_government_tax',
#'ef_government',
#'ef_money_currency',
#'ef_trade_tariffs_mean',
#'ef_trade_movement_visit']]
before_2016_data = data[data['year'] == 2016]
before_2016_data['year'].value_counts()
temp_data = before_2016_data[before_2016_data.isnull().sum(axis = 1) < 2]
temp_data.isnull().sum(axis = 1)

eemaldatavad_riigid = data.loc[(data["pf_rank"].isnull()) & (data["year"] == 2010)]["countries"].tolist() #Valime välja riigid millel puudub varasem info

eemaldatavad_aastad = [2008, 2009]
eemaldatavad_riigid = set(eemaldatavad_riigid) #Eemaldab kordused

for riik in eemaldatavad_riigid:
    data = data[~data["countries"].isin(eemaldatavad_riigid)] #Eemaldame riigid

for aasta in eemaldatavad_aastad:
    data = data[~data["year"].isin(eemaldatavad_aastad)] #Eemaldame aastad mille kohta meil on vähem infot

olemasolevad_riigid = set(data["countries"].tolist())

#data.isnull().sum(axis=1) #Puuduvad väärtused iga rea kohta
#data.info(verbose = True) #Puuduvad väärtused iga tulba kohta
pd.set_option('display.max_rows', None)
data.isnull().mean() * 100 # Mitu % andmetest on puudu
#data.loc[data["pf_expression_internet"].isnull()]

year                                   0.000000
ISO_code                               0.000000
countries                              0.000000
region                                 0.000000
pf_rol_procedural                     35.760971
pf_rol_civil                          35.760971
pf_rol_criminal                       35.760971
pf_rol                                 0.000000
pf_ss_homicide                         0.000000
pf_ss_disappearances_disap             0.653595
pf_ss_disappearances_violent           0.000000
pf_ss_disappearances_organized         6.816060
pf_ss_disappearances_fatalities        0.000000
pf_ss_disappearances_injuries          0.000000
pf_ss_disappearances                   0.000000
pf_ss_women_fgm                        6.722689
pf_ss_women_missing                    3.361345
pf_ss_women_inheritance_widows        16.339869
pf_ss_women_inheritance_daughters     16.339869
pf_ss_women_inheritance                2.521008
pf_ss_women                            1

#### Kood millega lisasin GDP teise datasetti
data_2 = pd.read_csv('.\datasets\Dataset2.csv')
gdp_data = pd.read_csv('.\datasets\gdp_data.csv')

gdp_data = gdp_data.drop(columns=['Country Code', 'Series Name', 'Series Code', '1990 [YR1990]', '2000 [YR2000]', '2019 [YR2019]', '2017 [YR2017]', '2018 [YR2018]'])

for year in range(2010, 2017, 1):
    print(year)
    gdp_data.rename(columns={(str(year) + " [YR" + str(year) + "]"): year}, inplace=True)

gdp_data = gdp_data.melt(id_vars=["Country Name"], 
        var_name="Year", 
        value_name="GDP")

#gdp_data = gdp_data.loc[gdp_data['Country Name'] in data_2['Country Name']]

gdp_data = gdp_data.reset_index()
for i in range(len(gdp_data)):
    if (gdp_data.loc[i]['Country Name']) not in set(data_2['Country Name']):
        gdp_data = gdp_data.drop(i, axis=0)




data_2 = data_2.merge(gdp_data, on=['Country Name', 'Year'], how='inner').drop(columns=['index'])
#data_2.to_csv("dataset2.csv", index=False)

In [3]:
data_2 = pd.read_csv('.\datasets\dataset2.csv')

before_2016_data = data_2[data_2['Year'] == 2008]
before_2016_data['Year'].value_counts()
temp_data = before_2016_data[before_2016_data.isnull().sum(axis = 1) < 2]
temp_data.isnull().sum(axis = 1)
data_2 = data_2[data_2["Country Name"].isin(olemasolevad_riigid)] #Valime samad riigid mida kasutame esimeses andmestikus
data_2.isnull().sum(axis=1) #Puuduvad väärtused iga rea kohta
data_2.info() #Puuduvad väärtused iga tulba kohta

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1072 entries, 1 to 2103
Data columns (total 12 columns):
Country Name                                           1072 non-null object
Expense (% of GDP)                                     800 non-null float64
Land area (sq. km)                                     1072 non-null float64
Life expectancy at birth, total (years)                1072 non-null float64
Military expenditure (% of GDP)                        1012 non-null float64
Population density (people per sq. km of land area)    1072 non-null float64
Population, female                                     1072 non-null float64
Population, male                                       1072 non-null float64
Population, total                                      1072 non-null float64
Revenue, excluding grants (% of GDP)                   816 non-null float64
Year                                                   1072 non-null int64
GDP                                                